# **Task 1: Data Preprocessing**

1.Print Out Sample Reviews

*  Load the dataset reviews_data.csv and display the first few rows to examine the reviews.

*  Print a few more samples if necessary to get a good understanding of the data.



In [144]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
import os
os.chdir("/content/drive/MyDrive")

In [146]:
#Importing libraries
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
import pickle
from sklearn.linear_model import Lasso, Ridge, LogisticRegression,SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [147]:
#Load the dataset reviews_data.csv and display the first few rows to examine the reviews
df = pd.read_csv("reviews_data.csv")

In [148]:
#Display first few rows using head()
df.head()

,name,location,Date,Rating,Review,Image_Links
0,Helen,"Wichita Falls, TX","Reviewed Sept. 13, 2023",5.0,Amber and LaDonna at the Starbucks on Southwes...,['No Images']
1,Courtney,"Apopka, FL","Reviewed July 16, 2023",5.0,** at the Starbucks by the fire station on 436...,['No Images']
2,Daynelle,"Cranberry Twp, PA","Reviewed July 5, 2023",5.0,I just wanted to go out of my way to recognize...,['https://media.consumeraffairs.com/files/cach...
3,Taylor,"Seattle, WA","Reviewed May 26, 2023",5.0,Me and my friend were at Starbucks and my card...,['No Images']
4,Tenessa,"Gresham, OR","Reviewed Jan. 22, 2023",5.0,I’m on this kick of drinking 5 cups of warm wa...,['https://media.consumeraffairs.com/files/cach...


In [149]:
#Display data attributes using describe()
df.describe()

,Rating
count,705.000000
mean,1.870922
std,1.397672
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,5.000000


2.Determine Pre-processing Techniques

* Identify the preprocessing techniques you will use:
* Remove stop words
* Lemmatization




In [150]:
df.sample(20)

,name,location,Date,Rating,Review,Image_Links
96,Izzy,"Louisville, KY","Reviewed Dec. 22, 2021",5.0,Incredible experience every time. Thankful for...,['No Images']
130,Ron,"Vancouver, BC","Reviewed Nov. 21, 2020",1.0,"On, November 21, 2020 my friend ordered a Vent...",['No Images']
747,Soheila,"Laguna Beach, CA","Reviewed April 1, 2010",NaN,I am a resident of Laguna Beach California for...,['No Images']
274,Charles,"Caronort, Saskatchewan","Reviewed Dec. 26, 2017",4.0,I used to visit Starbucks regularly on my way ...,['No Images']
521,Bebesh,"San Diego, CA","Reviewed April 20, 2015",1.0,Service at Birdrock La Jolla Starbucks - Hi gu...,['No Images']
607,Araceli,"Mission, TX","Reviewed Dec. 30, 2013",1.0,"The other day, Dec. 28th 2013, my son and I bo...",['No Images']
756,Mary,"Los Angeles, CA","Reviewed Jan. 6, 2010",NaN,My first day with the new gold card was so far...,['No Images']
676,Deborah,"Bellevue, WA","Reviewed Feb. 18, 2012",1.0,I just recently moved to Renton Washington and...,['No Images']
411,Margo,"Charlottesville, WI","Reviewed Aug. 20, 2016",3.0,"I enjoy Starbucks coffee, food, and other beve...",['No Images']
774,Sophie,"Quebec, QC","Reviewed Aug. 14, 2009",NaN,"This morning around 12h15 (Spain hour), I went...",['No Images']


In [151]:
df.isna().sum()

name             0
location         0
Date             0
Rating         145
Review           0
Image_Links      0
dtype: int64

In [152]:
df.shape

(850, 6)

In [153]:
imputer=SimpleImputer(strategy='most_frequent')
df['Rating']=imputer.fit_transform(df[['Rating']]).ravel()

In [154]:
df.isna().sum()

name           0
location       0
Date           0
Rating         0
Review         0
Image_Links    0
dtype: int64

In [155]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [156]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(words)

In [157]:
#Print all stop words
stop_words = set(stopwords.words('english'))
print("Stop words:", stop_words)

Stop words: {'own', 'had', 'will', 'he', 'couldn', 'nor', 'its', 'am', 'an', 're', 'aren', 'until', 'at', 'of', 'each', 'about', 'doesn', 'and', 'isn', 'can', 'only', 'as', 'd', 'his', 'ma', 'o', 'wouldn', 'most', "don't", 'is', 'you', 'those', 'any', 'weren', 'my', 'than', 'so', 'her', "you'd", "should've", 'ourselves', 'itself', "that'll", 'both', "weren't", 'him', 'again', 'wasn', 'if', 'herself', 'few', 'there', 'where', 'me', 'or', 'during', 'from', 'should', "couldn't", 'won', 'same', 'while', 'hasn', 'our', 'who', "wouldn't", 'shan', 'been', 'before', 'does', 'doing', 't', 'a', 'out', 'hers', "you're", "mightn't", 'too', 've', 'themselves', "shouldn't", 'yours', 'being', 'through', 'the', "hadn't", 'haven', 'm', 'hadn', "needn't", 'has', 'shouldn', 'because', 'not', 'no', "hasn't", 'ain', 'between', 'i', 'whom', 'under', 'then', "mustn't", 'with', "didn't", 'below', 'into', 'do', 'theirs', 'for', 'this', 'now', "it's", 'don', "won't", 'in', 'to', 'yourselves', 'myself', "you've"

3.Generate Cleaned Version of Text

* Apply the chosen preprocessing techniques to the review text.
* Store the cleaned text in a new column named processed_text.



In [158]:
df['processed_text'] = df['Review'].apply(preprocess_text)

# Check if 'processed_text' column exists after preprocessing
if 'processed_text' not in df.columns:
    raise ValueError("Column 'processed_text' not found. Perform data preprocessing first.")

# Display the first few rows to see the processed text
df[['Review', 'processed_text']].head()

,Review,processed_text
0,Amber and LaDonna at the Starbucks on Southwes...,amber ladonna starbucks southwest parkway alwa...
1,** at the Starbucks by the fire station on 436...,starbucks fire station 436 altamonte spring fl...
2,I just wanted to go out of my way to recognize...,wanted go way recognize starbucks employee bil...
3,Me and my friend were at Starbucks and my card...,friend starbucks card work thankful worker pai...
4,I’m on this kick of drinking 5 cups of warm wa...,kick drinking 5 cup warm water work instacart ...


4.Vectorization

* Use CountVectorizer or TF-IDF to convert the processed text into word-document matrix




In [159]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus
X_tfidf = tfidf_vectorizer.fit_transform(df['processed_text'])

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print(tfidf_df.head())

# Display results
print("Shape of the matrix:", X_tfidf.shape)

   0403907  06486   09   10  100  1000  1005  10254  102nd  105  ...  yous  \
0      0.0    0.0  0.0  0.0  0.0   0.0   0.0    0.0    0.0  0.0  ...   0.0   
1      0.0    0.0  0.0  0.0  0.0   0.0   0.0    0.0    0.0  0.0  ...   0.0   
2      0.0    0.0  0.0  0.0  0.0   0.0   0.0    0.0    0.0  0.0  ...   0.0   
3      0.0    0.0  0.0  0.0  0.0   0.0   0.0    0.0    0.0  0.0  ...   0.0   
4      0.0    0.0  0.0  0.0  0.0   0.0   0.0    0.0    0.0  0.0  ...   0.0   

   youth  youtube   yr  yuba  yukon  yummy  zeeb  zero  zip  
0    0.0      0.0  0.0   0.0    0.0    0.0   0.0   0.0  0.0  
1    0.0      0.0  0.0   0.0    0.0    0.0   0.0   0.0  0.0  
2    0.0      0.0  0.0   0.0    0.0    0.0   0.0   0.0  0.0  
3    0.0      0.0  0.0   0.0    0.0    0.0   0.0   0.0  0.0  
4    0.0      0.0  0.0   0.0    0.0    0.0   0.0   0.0  0.0  

[5 rows x 5216 columns]
Shape of the matrix: (850, 5216)


# **Task 2: Sentiment Analysis**

1.Select a Pre-trained Sentiment Analysis Model Implementation

* Choose a sentiment analysis model (e.g., VADER, TextBlob)

In [160]:
pip install vaderSentiment

In [161]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# 2. Generate Sentiment Score
# Apply the sentiment analysis model to the processed text
df['sentiment_score'] = df['processed_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

2.Generate Sentiment Score


* Apply the sentiment analysis model to the processed text.
* Store the sentiment scores in a new column named sentiment_score.

In [162]:
# Display the first few rows to see the sentiment scores
df[['processed_text', 'sentiment_score']].head()

,processed_text,sentiment_score
0,amber ladonna starbucks southwest parkway alwa...,0.8779
1,starbucks fire station 436 altamonte spring fl...,0.9670
2,wanted go way recognize starbucks employee bil...,0.4215
3,friend starbucks card work thankful worker pai...,0.9623
4,kick drinking 5 cup warm water work instacart ...,0.9723


# **Task 3: Topic Modeling**



1.Perform Topic Modeling

* Use the vectorized documents to perform topic modeling (e.g., LDA).

2.Print Most Frequent Words for Each Topic

* Describe your observations




In [163]:
num_topics = 5  # Number of topics to identify
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_matrix = lda.fit_transform(X_tfidf)

topic_words = lda.components_

In [164]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx + 1}: "
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

n_top_words = 10
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lda, tfidf_feature_names, n_top_words)

Topic #1: starbucks coffee drink said time one card order like went
Topic #2: starbucks coffee customer service card order time drink get one
Topic #3: text review starbucks customer store service one coffee order like
Topic #4: organic prepare feta natatia example ridiculous rudely pistachio cookie pain
Topic #5: starbucks coffee drink customer store time get always good go



# **Task 4: Prediction**


1.Train a Classifier to Predict Rating

* Use all the information from previous steps (processed text, sentiment scores, topic modeling) to train a classifier predicting the rating of the review.

* Save the prediction result to a new column named predicted_rating.

2.Evaluate the Model

* Discuss the accuracy of the model and its implications.







In [165]:
X_features = np.hstack((X_tfidf.toarray(), df['sentiment_score'].values.reshape(-1, 1), lda_matrix))
y = df['Rating'].values

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [167]:
# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [168]:
# Predict Ratings and Save Predictions to DataFrame
df['predicted_rating'] = clf.predict(X_features)

In [171]:
# Evaluate the Model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Display evaluation results
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:",conf_matrix)

# Save the updated DataFrame with predictions
df.to_csv('/content/drive/MyDrive/reviews_data_with_predictions.csv', index=False)
print("Predictions saved to 'reviews_data_with_predictions.csv'.")

Accuracy: 0.6941176470588235

Classification Report:
               precision    recall  f1-score   support

         1.0       0.69      1.00      0.81       110
         2.0       0.00      0.00      0.00        19
         3.0       0.00      0.00      0.00         5
         4.0       0.00      0.00      0.00        10
         5.0       0.80      0.31      0.44        26

    accuracy                           0.69       170
   macro avg       0.30      0.26      0.25       170
weighted avg       0.57      0.69      0.60       170


Confusion Matrix: [[110   0   0   0   0]
 [ 19   0   0   0   0]
 [  5   0   0   0   0]
 [  8   0   0   0   2]
 [ 18   0   0   0   8]]
Predictions saved to 'reviews_data_with_predictions.csv'.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [170]:
# The model correctly predicts the rating 69.41% of the time. While this seems reasonable, accuracy alone doesn't provide a complete picture, especially with class imbalance.
# The macro average provides an average of the metrics for each class, treating all classes equally. The low values indicate that the model performs poorly across most classes.
# The weighted average accounts for the number of instances in each class. Since class 1 dominates the dataset, the weighted metrics are influenced heavily by the performance on this class. The precision, recall, and F1-score are moderate but not excellent.